# Ансамбли

1111


In [1]:
import sys

from tensorflow.keras import datasets, layers, models, losses
import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
import matplotlib.pyplot as plt
from livelossplot import PlotLossesKeras
import tensorflow_addons as tfa

sys.path.append('../../../')

from core.datasets import get_ds
from core.make_answer import make_ans_file

C:\Users\Salam\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
transformer = tv.transforms.Compose([
    tv.transforms.RandomAdjustSharpness(sharpness_factor=5, p=.4),
    tv.transforms.RandomHorizontalFlip(p=.5),
    tv.transforms.RandomPosterize(bits=3, p=.3),
    tv.transforms.RandomPerspective(distortion_scale=0.2, p=.5),
    tv.transforms.RandomRotation(degrees=(-10, 10)),
    tv.transforms.ColorJitter(brightness=.2, hue=0.2),
])

In [3]:
train_ds, val_ds, test_ds = get_ds('repaired_data_train', 'repaired_data_test', transform=transformer, one_hot=True, back=3)

In [4]:
model_1 = tf.keras.models.load_model(f'../../../models/checkpoints/model1_categorical_accuracy.h5')
model_3 = tf.keras.models.load_model(f'../../../models/checkpoints/model3_categorical_accuracy.h5')
model_C_EN_1 = tf.keras.models.load_model(f'../../../models/checkpoints/model_C_EN_1_categorical_accuracy.h5')

- Bagging (пакетирование) голосование, предпочтительна система наподобие земских соборов с представителями разных сословий нейронных сетей.

In [25]:
def get_bagging_pred(models, x):
    pred_summ = np.full((len(x), 10), 0.)
    for m in models:
        pred_summ += m.predict(x,  verbose=False)
    pred_summ = np.array([np.argmax(i) for i in pred_summ])
    return pred_summ

ensemble_acc_summ = 0
model_1_acc_summ = 0
model_3_acc_summ = 0
model_C_EN_1_acc_summ = 0
i = 0
for batch in train_ds:
    if i%10 == 0:
        print(i, "/", len(train_ds))
    i+=1
    y_true = np.array([np.argmax(i) for i in batch[1]])
    y_pred_ensemble = get_bagging_pred(models=[model_1, model_3, model_C_EN_1], x=batch[0])
    y_pred_model_1 = np.array([np.argmax(i) for i in model_1.predict(batch[0], verbose=False)])
    y_pred_model_3 = np.array([np.argmax(i) for i in model_3.predict(batch[0], verbose=False)])
    y_pred_model_C_EN_1 = np.array([np.argmax(i) for i in model_C_EN_1.predict(batch[0], verbose=False)])
    model_1_acc_summ += np.sum(y_true == y_pred_model_1) / 512
    model_3_acc_summ += np.sum(y_true == y_pred_model_3) / 512
    model_C_EN_1_acc_summ += np.sum(y_true == y_pred_model_C_EN_1) / 512
    ensemble_acc_summ += np.sum(y_true == y_pred_ensemble) / 512

print("model_1", model_1_acc_summ / 91)
print("model_3", model_3_acc_summ / 91)
print("model_C_EN_1", model_C_EN_1_acc_summ / 91)
print("ensemble", ensemble_acc_summ / 91)

# mmm = MulticlassClassificationMetrics(y_true=y_true, y_pred=y_pred)
# print(mmm.accuracy())

0 / 91
10 / 91
20 / 91
30 / 91


KeyboardInterrupt: 